In [6]:
import numpy as np
import torch
import pyvista as pv
pv.set_jupyter_backend('static')
import plotly.graph_objects as go
import sys
sys.path.insert(0, r'C:\Users\Noahc\Documents\USYD\PHD\8 - Github\Rev909')
from utils.mesh_utils import BatchedAngularMeshRavel

In [7]:
dataset_path = r'C:\Users\Noahc\Documents\USYD\PHD\0 - Work Space\Markov Studies v2\RAS_Re100k.npy'
mesh_path = r"C:\Users\Noahc\Documents\USYD\tutorial\cylinder_mno_turb5\case.foam"

coord_points = np.load(dataset_path[:-4]+'_coords.npy')
dataset = np.load(dataset_path)
Ravler = BatchedAngularMeshRavel(coord_points,m=59,n=356)
dataset.shape

(901, 21004, 3)

In [9]:
data = Ravler.to(torch.tensor(dataset[...,:2], dtype=torch.float32), forward=True)
T, W, H, C = data.shape

T_in = 0
split = 0.9
sub = 1

T = T - T_in
print(f'Raw Dataset has shape:', data.shape, f'with shape {W:n}x{H:n}')
ntrain = int(split*T)
ntest = T - ntrain
print(f'Dataset is split for training ({100*split:.0f}%) to {ntrain:n} - {ntest:n} with downsample of {sub:n}x')

W = int(W/sub) 
H = int(H/sub) 
data = data[T_in:, ::sub, ::sub, :]

max_norm = np.linalg.norm(data[:,T_in:,...], axis=(2, 3)).max()
print(max_norm)
max_norm = np.linalg.norm(data[T_in:,...], axis=(1, 2)).max()
print(max_norm)

Raw Dataset has shape: torch.Size([901, 59, 356, 2]) with shape 59x356
Dataset is split for training (90%) to 810 - 91 with downsample of 1x
21.97066
146.39618


In [127]:
max_norm = np.linalg.norm(data[...,:], axis=(1, 2)).max(axis=0)#.max(axis=0)
print(max_norm)

[146.39618  34.69926]


In [128]:
all(isinstance(x, np.ndarray) for x in [max_norm, max_norm*4])

True

In [125]:
isinstance([max_norm, max_norm*4], (list, tuple))

True

In [120]:
[max_norm, max_norm*4]

[array([146.39618,  34.69926], dtype=float32),
 array([585.5847 , 138.79704], dtype=float32)]

In [133]:
[max_norm, max_norm*4]#[0]

[array([146.39618,  34.69926], dtype=float32),
 array([585.5847 , 138.79704], dtype=float32)]

In [110]:
dataset.shape

(901, 21004, 3)

In [151]:
max_norm = np.linalg.norm(dataset[T_in:,...,:2], axis=(1)).max()
print(max_norm)

146.39621


In [154]:
from dissipative_utils import sample_uniform_spherical_shell, linear_scale_dissipative_target
class RegulatorSampler():
    def __init__(self, 
                 radii, 
                 shape, 
                 scale_down_factor = 0.5, 
                 weight = 0.01, 
                 sampling_fn = sample_uniform_spherical_shell,
                 target_fn = linear_scale_dissipative_target,
                 loss_fn=torch.nn.MSELoss(reduction='mean')):
        
        self.sampling_fn = sampling_fn
        self.target_fn = target_fn
        self.loss_function = loss_fn
        self.weight = weight
        self.scale_down_factor = scale_down_factor
        
        if all(isinstance(x, np.ndarray) for x in radii):
            assert len(radii[0]) == shape[-1]
            print('Regularizer enforces per channel')
            self.split_channels = True 
            radii = np.concatenate([max_norm[None,...], max_norm[None,...]*4], axis = 0)
        else:
            self.split_channels = False 

        # get shape((S, S, 1))
        self.shape = shape
        # get radii
        self.radii = radii

    def get_input(self, batch_s):
        if self.split_channels:
            input_sample = []
            for i in range(self.shape[-1]):
                input_sample.append(torch.tensor(self.sampling_fn(batch_s, self.radii[...,i], self.shape[:-1]+ (1,)), dtype=torch.float))
            input_sample = torch.concatenate(input_sample, dim=-1)
        else:
            input_sample = torch.tensor(self.sampling_fn(batch_s, self.radii, self.shape), dtype=torch.float)
        return input_sample
    
    def get_target(self, x_diss):
        return self.target_fn(x_diss, self.scale_down_factor)
    
    def loss_fn(self, out, y):
        return self.weight * self.loss_function(out, y)

In [30]:
max_norm

array([146.39618,  34.69926], dtype=float32)

In [39]:
sample_uniform_spherical_shell(4,[max_norm, max_norm*4],(W, H, 2)).shape

(4, 59, 356, 2)

In [59]:
max_norm

array([146.39618,  34.69926], dtype=float32)

In [134]:
max_norm_2 = np.concatenate([max_norm[None,...], max_norm[None,...]*4], axis = 0)
print([max_norm, max_norm*4])
print(max_norm_2)
print(max_norm_2[...,0])

[array([146.39618,  34.69926], dtype=float32), array([585.5847 , 138.79704], dtype=float32)]
[[146.39618  34.69926]
 [585.5847  138.79704]]
[146.39618 585.5847 ]


In [ ]:
x = data[:4, ..., :]

In [155]:
sampling_class = RegulatorSampler(radii=[max_norm, max_norm*4], shape=(W, H, 2), scale_down_factor=0.5, weight=0.01)

In [136]:
x_diss = sampling_class.get_input(x.shape[0])
x_diss.shape

torch.Size([4, 59, 356, 2])

In [157]:
np.linalg.norm(x_diss, axis=(1, 2)).max(axis=0)

array([341.09988, 342.80563], dtype=float32)

In [158]:
np.linalg.norm(y_diss, axis=(1, 2)).max(axis=0)

array([170.54994, 171.40282], dtype=float32)

In [156]:
x_diss = sampling_class.get_input(x.shape[0])
assert(x_diss.shape == x.shape)
y_diss = sampling_class.get_target(x_diss)